In [17]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression
import numpy as np

In [12]:
# Data loading and basic trasformations
import tflearn.datasets.mnist as mnist
X, Y, X_test, Y_test = mnist.load_data(one_hot=True)
X = X.reshape([-1,28,28,1])
X_test = X_test.reshape([-1,28,28,1])

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [13]:
# Building the network
CNN = input_data(shape=[None, 28, 28, 1], name='input')                 # Input
CNN = conv_2d(CNN, 32, 5, activation='relu', regularizer="L2")          # 합성곱 계층1
CNN = max_pool_2d(CNN, 2)                                               # Pooling 1
CNN = local_response_normalization(CNN)                                 # LRN 처리 1(local response normalization)
CNN = conv_2d(CNN, 64, 5, activation='relu', regularizer="L2")          # 합성곱 계층2
CNN = max_pool_2d(CNN, 2)                                               # Pooling 2
CNN = local_response_normalization(CNN)                                 # LRN 처리 2(local response normalization)
CNN = fully_connected(CNN, 1024, activation=None)                       # 완전연결계층
CNN = dropout(CNN, 0.5)                                                 # Drop Out
CNN = fully_connected(CNN, 10, activation='softmax')                    # 판독 계층
CNN = regression(CNN, optimizer='adam', learning_rate=0.0001,           # 회귀모형 모델
                     loss='categorical_crossentropy', name='target')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [14]:
# 네트워크 학습
model = tflearn.DNN(CNN, tensorboard_verbose=0,
                   tensorboard_dir = 'MNIST_tflearn_board/',
                   checkpoint_path = 'MNIST_tflearn_checkpoint/checkpoint')
model.fit({'input': X}, {'target': Y}, n_epoch=3, 
           validation_set=({'input': X_test}, {'target': Y_test}),
           snapshot_step=1000,show_metric=True, run_id='convnet_mnist')

Training Step: 2579  | total loss: 0.04970 | time: 160.532s
| Adam | epoch: 003 | loss: 0.04970 - acc: 0.9863 -- iter: 54976/55000
Training Step: 2580  | total loss: 0.05050 | time: 168.781s
| Adam | epoch: 003 | loss: 0.05050 - acc: 0.9845 | val_loss: 0.04389 - val_acc: 0.9857 -- iter: 55000/55000
--
INFO:tensorflow:/home/younseun/columbus_study/04_learning_tensorflow/07__abstractions/MNIST_tflearn_checkpoint/checkpoint-2580 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/home/younseun/columbus_study/04_learning_tensorflow/07__abstractions/MNIST_tflearn_checkpoint/checkpoint-2580 is not in all_model_checkpoint_paths. Manually adding it.


In [15]:
evaluation = model.evaluate({'input': X_test},{'target': Y_test})
print(evaluation)

[0.9857]


In [19]:
pred = model.predict({'input': X_test})
print((np.argmax(Y_test,1)==np.argmax(pred,1)).mean())

0.9857


In [20]:
Y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
np.argmax(Y_test,1)

array([7, 2, 1, ..., 4, 5, 6])